In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,163 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [5]:
# Filter data and create a new DataFrame to retrieve all rows where total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [6]:
# Filter total_votes_df and create a new DataFrame to retrieve all rows where number of helpful_votes dived by total_votes is equal to or greater than 50%.
helpfulvotes_percent_df = total_votes_df.withColumn('percentage_votes',col('helpful_votes')/col('total_votes')).filter(col("percentage_votes") >= .50)
helpfulvotes_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percentage_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|0.8372093023255814|
|R3A1VQ3SQDXEJ3|          5|  

In [7]:
# Filter helpfulvotes_percent_df and create a new DataFrame that retrieves all rows where a review was written as a part of the Vine program.
paid_vine_df = helpfulvotes_percent_df.filter(col("vine") == "Y").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percentage_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|               0.9|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|               1.0|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|0.9285714285714286|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|0.9666666666666667|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|               1.0|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|0.9615384615384616|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|              0.96|
|R25FI3J7WWOYXM|          3|  

In [8]:
# Filter helpfulvotes_percent_df and create a new DataFrame that retrieves all rows where a review was written as NOT a part of the Vine program.
unpaid_vine_df = helpfulvotes_percent_df.filter(col("vine") == "N").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percentage_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|0.8372093023255814|
|R3A1VQ3SQDXEJ3|          5|  

In [10]:
# Determine the total number of reivews, total number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
from pyspark.sql.functions import col,when,count,lit
reviews_total_df = helpfulvotes_percent_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total")
).show()

+----+-------------+--------------------+------------------+
|vine|Total_Reviews|Total_5_Star_Reviews| %_5_Star_To_Total|
+----+-------------+--------------------+------------------+
|   Y|          170|                  65| 38.23529411764706|
|   N|        37840|               20612|54.471458773784356|
+----+-------------+--------------------+------------------+

